<a href="https://colab.research.google.com/github/djbell01/563-DawsonBell/blob/main/MC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class Particle():
  def __init__(self,position,r):
    self.position = np.array(position)
    self.radius = r

class System():
  def __init__(self,N,L,particle_radius,r_cut):
    self.N = N
    self.L = L
    self.r_cut = r_cut

    self.particles = []
    for i in range(self.N):
      position = np.random.uniform(0,self.L,3)
      self.particles.append(Particle(position,particle_radius))

    # Check and resolve overlaps
    for i in range(self.N):
        overlap = True
        while overlap:
            overlap = False
            for j in range(i):  # Only check previous particles
                distance = self.minimum_image_distance(self.particles[i].position, self.particles[j].position)
                if distance < 2 * particle_radius:  # Overlap detected
                    self.particles[i].position = np.random.uniform(0, self.L, 3)
                    overlap = True
                    break  # Restart check for this particle

  def minimum_image_distance(self,pos1,pos2):
    diff_vector = pos1 - pos2
    diff_vector -= self.L * np.round(self.L/diff_vector)
    return np.linalg.norm(diff_vector)

  def calculate_energy(self):
    energy = 0
    for i in range(self.particles):
      for j in range(self.particles):
        if i < j:
          distance = self.minimum_image_distance(self.particles[i].position,self.particles[j].position)
          energy += 4*((self.particles[i].radius/distance)**12 - (self.particles[i].radius/distance)**6)


In [50]:
s = System(10,10,0.5,3)
for particle in s.particles:
  print(particle.position)

[6.15015699 0.48263074 3.66412726]
[1.10995253 5.33675207 3.33109052]
[1.19841325 1.23963046 3.45897079]
[0.44877514 1.85691679 8.40788021]
[0.18240281 5.61427625 3.66022289]
[9.72480895 8.13498113 1.04292244]
[1.8909862 7.2567821 6.9578327]
[4.87648046 9.99614288 4.25030637]
[1.96508619 4.34775943 8.73693401]
[3.27231957 7.85748931 1.82029977]
